In [1]:
import random
import math
from Crypto.Util import number
import multiprocessing
from joblib import Parallel, delayed

In [2]:
class keyGen:
    def __init__(self,nbits):
        self.q = number.getPrime(nbits)
        self.alpha = self.getalpha()
        self.privateKey = random.randrange(1,10,1)
    
    def getalpha(self):
        alpha = -1
        while alpha == -1:
            alpha = self.getPrimitiveRoot(self.q)
        return alpha
    
    def power(self, x, y, p):  
        res = 1 # Initialize result  
        x = x % p # Update x if it is more  
                  # than or equal to p  
        while (y > 0):  
            # If y is odd, multiply x with result  
            if (y & 1): 
                res = (res * x) % p  
            # y must be even now  
            y = y >> 1 # y = y/2  
            x = (x * x) % p  
        return res  

    def getPrimeFactors(self,phi):
        factors = set()
        while (phi % 2 == 0) : 
            factors.add(2)  
            phi = phi // 2

        #when phi is odd
        for i in range(3, int(math.sqrt(phi)), 2): 
            while (phi % i == 0) : 
                factors.add(i)  
                phi = phi // i  

        # This condition is to handle the case  
        # when phi is a prime number greater than 2  
        if (phi > 2) : 
            factors.add(phi)  

        return factors
    
    def getPrimitiveRoot(self,prime):
        phi = prime - 1
        factors = self.getPrimeFactors(phi)

        ret_list = Parallel(n_jobs = multiprocessing.cpu_count())(delayed(self.checkPrimitivePrime)(i, prime, factors) for i in range(2, phi+1))
        ret_list = list(filter(None, ret_list))
        
        return random.choice(ret_list)
    
    def checkPrimitivePrime(self,i,q,factors):
        phi = q - 1
        for factor in factors:
            if self.power(i, phi // factor, q) == 1:
                return None
        return i

In [3]:
keyA = keyGen(20)

In [8]:
import socket               # Import socket module

s = socket.socket()         # Create a socket object
host = socket.gethostname() # Get local machine name
port = 8002                 # Reserve a port for your service.
s.bind((host, port))        # Bind to the port

s.listen(5)                 # Now wait for client connection.
while True:
    c, addr = s.accept()     # Establish connection with client.
    print('Got connection from', addr)
    c.send(bytes(str(keyA.q),'utf8'))
    c.send(bytes(str(keyA.alpha),'utf8'))
    c.send(bytes(str(publicKeyA),'utf8'))
    publicKeyB = c.recv(1024)
    c.close() 
    break

Got connection from ('127.0.0.1', 43596)


In [9]:
publicKeyB = str(publicKeyB,'utf8')
publicKeyB = int(publicKeyB)
secretKey = (publicKeyB**keyA.privateKey)%keyA.q
secretKey

550502